# Scraping beer review site 

In [1]:
# Import dependencies for data wrangling  
import pandas as pd 
import numpy as np
import requests 
import json
from pprint import pprint
from bs4 import BeautifulSoup
import requests
import pymongo

In [2]:
# Import dependencies for scraping
from time import sleep

from splinter import Browser 
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
# Open the Chrome Driver Browser
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [C:\Users\pooja\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache


In [4]:
# Read in CSV
beer_df = pd.read_csv("../../Data/beer.gz", encoding="ISO-8859-1")
beer_df.head()

,Unnamed: 0,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,0,10325,Vecchio Birraio,2009-02-16 20:57:03,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,1,10325,Vecchio Birraio,2009-03-01 13:44:57,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,2,10325,Vecchio Birraio,2009-03-01 14:10:04,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,3,10325,Vecchio Birraio,2009-02-15 19:12:25,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,4,1075,Caldera Brewing Company,2010-12-30 18:53:26,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [5]:
# Drop the "unnamed: 0" column 
beer_df = beer_df.drop(['Unnamed: 0'], axis = 1)
beer_df.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,2009-02-16 20:57:03,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,2009-03-01 13:44:57,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,2009-03-01 14:10:04,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,2009-02-15 19:12:25,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,2010-12-30 18:53:26,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [6]:
beer_names = beer_df["beer_beerid"].unique()
type(beer_names)

numpy.ndarray

In [7]:
# Page URL that we are going to scrape
url = "https://www.beeradvocate.com/beer/profile/" + str(beer_names[0]) + "/"
print(url)
browser.visit(url) 

# Convention to let the script we are running sleep to allow everything to load 
sleep(0.5)

https://www.beeradvocate.com/beer/profile/47986/


In [48]:
# Create a BeautifulSoup object and parse as lxml
html = browser.html
soup = BeautifulSoup(html, 'lxml')

In [62]:
import re

In [80]:
results = soup.find("div", class_ = "break")
clean_results = results.text.split()
clean_results[3:11]

['4814',
 'Hardware',
 'Dr',
 'NEAlbuquerque,',
 'New',
 'Mexico,',
 '87109United',
 'States(505)']

In [81]:
clean_results[9]

'87109United'

In [82]:
r = re.compile("([0-9]+)([a-zA-Z]+)")
m = r.match(text)
m.group(1)

'87109'

In [40]:
def listToString(s):
    
    address_string = ""
    
    for word in s:
        address_string += " " + word
    return address_string

s = address
print(listToString(s))  

 4814 Hardware Dr NEAlbuquerque, New Mexico, 87109United States(505)


In [20]:
x_path = ""

name = browser.find_by_xpath(x_path).text

ElementDoesNotExist: no elements could be found with xpath ""

In [ ]:
# Interact with the map link
browser.click_link_by_partial_text('map')

In [ ]:
# Obtain the info box that contains address 
results = soup.find_all("div", class_ = "section-hero-header_title_description")

In [ ]:
https://www.google.com/maps/place/4814+Hardware+Dr+NE,+Albuquerque,+NM+87109,+USA/@35.1364451,-106.5916661,17z/data=!4m5!3m4!1s0x872274e43d349b1d:0x2ba0c944a0168f47!8m2!3d35.1364451!4d-106.5894774

In [ ]:
##//*[@id="info_box"]/a[4]

In [ ]:
##browser.find_by_xpath('//*[@id="info_box"]/a[4]')

In [ ]:
# Obtain the info box that contains address 
results = soup.find("div", class_="break")
print(results.prettify())